## MongoDB連線

In [1]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['test2']

## Search + Jieba

In [7]:
import re
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

#給關鍵字----------------------------------------
tag1 = "柯文哲"
tag2 = "雙塔"
#-----------------------------------------------

date=[]
title=[]
content =[]
#把資料庫東西抓出來
for post in collection.find(
    {"$and":[                   
            {"tfidf":{"$regex":tag1}},
            {"tfidf":{"$regex":tag2}},
            #{"date":{"$regex":"2016"}},
            ]},{"_id":0}): 
    summary = post['content']
    content.append(' '.join(jieba.cut(summary)))
    title.append(post['title'])
    date.append(post['date'])

print title[0]

#文章數
print "文章數:",len(title)

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.579 seconds.
Prefix dict has been built succesfully.


周六拼「一日雙塔」,柯將缺席228紀念
文章數: 7


## TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)  # titile 放文本
weight = X.toarray()

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Feature_num 762
[[ 0.          0.          0.         ...,  0.08691511  0.08053426  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.04529932  0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.02600163  0.          0.04818545]
 [ 0.          0.          0.05727385 ...,  0.03723214  0.          0.        ]]


## Get TOP TF-IDF

In [24]:
# 抓出TOP TF-IDF 

indices = np.argsort(X.toarray()[0])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
features = vectorizer.get_feature_names()     # vectorizer = CountVectorizer()

#要看前幾名的權重
top_n = 100
top_features = [features[i] for i in indices[:top_n]]

a=0
for i in top_features:
    print i,weight[0][indices[a]]
    a=a+1
    
#單獨看前幾名的權重值    
print indices
print weight[0][indices[0]]
print weight[0][indices[1]]

高雄 0.264653264044
台61 0.21489576309
周六 0.176597262025
打氣 0.154745909422
雲林 0.146709142477
台北 0.135460848368
楠梓 0.129966348716
百八十 0.12522026791
有網友 0.121207513765
候選人 0.111922774565
單車挑戰 0.111922774565
北市 0.107456391972
01 0.10717669376
立委候選人 0.0996879338663
路線 0.0981097916048
騎單車 0.0979212180122
規劃 0.088070466964
凌晨 0.0816443597161
沿途 0.0808433311316
清晨 0.0785783053526
他本 0.0769093281585
十二位 0.0769093281585
大卡車 0.0769093281585
昨被 0.0769093281585
頂多 0.0769093281585
高潮 0.0769093281585
共三 0.0769093281585
柯曾 0.0769093281585
二世 0.0769093281585
竹圍漁港 0.0769093281585
數天 0.0769093281585
二十一 0.0769093281585
超馬 0.0769093281585
二時 0.0769093281585
月底 0.0769093281585
不難 0.0769093281585
騎三 0.0769093281585
大部分 0.0769093281585
channel 0.0769093281585
安南 0.0769093281585
搶鏡 0.0769093281585
平地 0.0769093281585
稱柯是 0.0769093281585
小山坡 0.0769093281585
新竹 0.0764946836174
15 0.075835626265
幕僚 0.073953020007
城市 0.0721974307398
媒體 0.071637594648
注意安全 0.07163192103
柯上 0.07163192103
五十七歲 0.07163192103
一個月 0.07163

## Clustering

In [16]:
from sklearn import cluster
c = cluster.KMeans(n_clusters=2)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點

In [19]:
for idx, grp in enumerate(k_data):
    if grp == 0:
        print title[idx],date[idx]
        print "-------------------------------"
    if grp == 1:
        print title[idx],date[idx]

周六拼「一日雙塔」,柯將缺席228紀念 20160225
-------------------------------
57歲拼520公里「勿輕易嘗試」 20160228
-------------------------------
太強了,太痛苦了,28小時騎完雙塔,柯：絕不再幹 20160229
-------------------------------
【空拍】柯P抵達鵝鑾鼻燈塔瞬間 20160229
-------------------------------
柯玩洋單車,歪歪扭扭 20160301
-------------------------------
柯文哲戰駒,今午1元起標雙城雙塔2車值37萬,標金捐作公益 20160304
戰駒飆74萬,柯允陪騎2分鐘 20160306


In [36]:
grp1_news = []
grp2_news = []
for idx, grp in enumerate(k_data):
    if grp == 0:
        news =  {
            'title':title[idx],
            'date':date[idx]
        }
        grp1_news.append(news)
    if grp == 1:
        news =  {
            'title':title[idx],
            'date':date[idx]
        }
        grp2_news.append(news)
print type(grp1_news)
print len(grp1_news)

<type 'list'>
5


## 全文切詞把重要的TF-IDF存進資料庫中

In [2]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['test']

In [3]:
import re
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

title=[]
content =[]
#把資料庫東西抓出來
for post in collection.find(): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    title.append(post['title'])

print title[2000]
print title[1]

#文章數
print "文章數:",len(title)

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.477 seconds.
Prefix dict has been built succesfully.


吳育昇開砲,「內閣定要改組」國營工會抗議砍年終,「誰要出來祭旗」
燦爛跨年,歡動全台
文章數: 7135


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)  # titile 放文本
weight = X.toarray()

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Feature_num 123652
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [33]:
# 抓出TOP TF-IDF 
# (X.toarray()[2000]) 填入文章編號(我要看第2000號的文章)
indices = np.argsort(weight[0])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
features = vectorizer.get_feature_names()     # vectorizer = CountVectorizer()


#印出該編號文章的標題
print title[0]

# top_n = 50 填入要看TOP多少的詞(我要看前50名)
top_n = 50
top_features = [features[i] for i in indices[:top_n]]

a=0
for i in top_features:
    print i,weight[0][indices[a]]  #weight[2000] 裡面請填入跟上面一樣的文章編號
    a=a+1
    
#單獨看前幾名的權重值    
print indices
print weight[0][indices[0]]

2013,台灣起舞
煙火 0.611499711968
施放 0.229148200446
大樓 0.223928031396
台北101 0.205642084524
萬發 0.138123338697
時代廣場 0.131524216952
煙火秀 0.129015347303
義大世界 0.116611185047
環繞 0.112763854427
搭配 0.095313843372
遊客 0.0869158483401
全場 0.0777828754576
法國廠商 0.0747023059167
氣勢十足 0.0747023059167
全亮 0.0747023059167
接著紅 0.0747023059167
跳著 0.0747023059167
搭配紅 0.0747023059167
由下而上 0.0747023059167
幸福家庭 0.0747023059167
全台 0.0737370614413
大樓外牆 0.0714027450441
549 0.0714027450441
夜空 0.0714027450441
塔尖 0.0714027450441
火鳥 0.0714027450441
光束 0.0714027450441
造型 0.0712170708926
大朵 0.0690616693485
祝福語 0.0690616693485
逗樂 0.0690616693485
繞著 0.0690616693485
華爾滋 0.0690616693485
北海小英雄 0.0690616693485
手語 0.0672457899642
劍湖山世界 0.0672457899642
爆開 0.0672457899642
亮出 0.0672457899642
古典樂 0.0672457899642
整棟 0.0672457899642
設計概念 0.065762108476
飛舞 0.065762108476
煙花 0.065762108476
明快 0.0645076736517
金龍 0.0645076736517
台灣史 0.0645076736517
地形 0.0645076736517
照亮 0.0634210327804
跑馬燈 0.0624625476034
570 0.0624625476034
[79936 64472 42128